In [4]:
import pandas as pd

In [3]:
df=pd.read_excel("filtered_data.xlsx")
print(df.shape)

(86261, 7)


In [35]:
columns=['Гүйлгээний огноо', 'Харилцагчийн код', 'Харилцагчийн нэр', 'Дебет - ₮', 'Кредит -₮',"Тооцооны төрөл" ]


avlaga_nasjilt=pd.read_excel("Авлага_насжилт.xlsx", usecols=columns)
print(avlaga_nasjilt.shape)
print(avlaga_nasjilt.columns)

(10328, 6)
Index(['Гүйлгээний огноо', 'Тооцооны төрөл', 'Харилцагчийн код',
       'Харилцагчийн нэр', 'Дебет - ₮', 'Кредит -₮'],
      dtype='object')


In [36]:
# Огноог datetime болгох
avlaga_nasjilt['Гүйлгээний огноо'] = pd.to_datetime(avlaga_nasjilt['Гүйлгээний огноо'], errors='coerce')
# avlaga_nasjilt['Гүйлгээний огноо'] = pd.to_datetime(avlaga_nasjilt['Гүйлгээний огноо'], format='%Y-%m-%d')
# avlaga_nasjilt['Гүйлгээний огноо'] = avlaga_nasjilt['Гүйлгээний огноо'].dt.strftime('%Y-%m-%d')
# Сар багана гаргах
avlaga_nasjilt['Сар'] = avlaga_nasjilt['Гүйлгээний огноо'].dt.to_period('M')

In [37]:
avlaga_nasjilt["Үлдэгдэл"] = avlaga_nasjilt['Дебет - ₮'] - avlaga_nasjilt['Кредит -₮']

In [38]:
pivot_sorted= avlaga_nasjilt.sort_values(['Харилцагчийн код', 'Харилцагчийн нэр', 'Тооцооны төрөл', 'Сар'])
pivot_sorted['Нийт үлдэгдэл'] = pivot_sorted.groupby(['Харилцагчийн код', 'Харилцагчийн нэр', 'Тооцооны төрөл',])['Үлдэгдэл'].cumsum()
latest_balances = pivot_sorted.groupby(['Харилцагчийн код', 'Харилцагчийн нэр', 'Тооцооны төрөл']).last().reset_index()
# pivot_table = avlaga_nasjilt.groupby(['Харилцагчийн код', 'Харилцагчийн нэр', 'Сар'])["Үлдэгдэл"].sum().reset_index()
latest_balances = latest_balances[latest_balances['Нийт үлдэгдэл'] != 0]

In [39]:
latest_balances.to_excel("latest_balances.xlsx", index=False)

In [26]:
last_c2=pivot_filtered.sort_values("Сар").groupby(['Харилцагчийн код', 'Харилцагчийн нэр']).tail(1)

In [27]:
last_c2.to_excel("pivot_filtered.xlsx", index=False)

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
df=pd.read_excel("ЕЖ.xlsx")
print(df.columns)


# pd.read_excel("ЕЖ.xlsx", usecols=['Гүйлгээний утга', 'Харилцагчийн нэр', 'Тооцооны төрөл', 'Сар', 'Нийт үлдэгдэл']).to_excel("ЕЖ.xlsx", index=False)

Index(['Дугаар', 'ДТ2', 'КТ2', 'ДТ4', 'КТ4', 'ДТ6', 'КТ6', 'сар, өдөр',
       'дугаар\n', 'Гүйлгээний утга', 'Дүн', 'Дебет', 'Кредит\n'],
      dtype='object')


In [18]:
df['Огноо'] = pd.to_datetime(df['сар, өдөр'])
# print(df["Дүн"].head(10))
debet_df = df[["Огноо", "Дебет", "Дүн"]].rename(columns={"Дебет": "Данс код"})
debet_df["Сар"] = debet_df["Огноо"].dt.to_period("M")
pivot = debet_df.groupby([ "Данс код"])["Дүн"].sum().reset_index()


In [ ]:
# Дүн баганыг тоон төрөлд хөрвүүлэх
pivot['Дүн'] = pd.to_numeric(pivot['Дүн'], errors='coerce')

# График зурна
import matplotlib.pyplot as plt
import seaborn as sns

print(pivot["Дүн"].dtype)
print(pivot["Дүн"].head(10))
print("PN тооцоо.xlsx")
# print(pivot["Сар"].dtype)
plt.figure(figsize=(14, 6))
# pivot['Сар'] = pivot['Сар'].astype(str)  # Сар баганыг string төрөлд хөрвүүлэх
pivot["Дүн"]=pivot["Дүн"].map('{:.2f}'.format)  # Дүн баганыг форматлах
print(pivot["Дүн"].head(10))
sns.lineplot(data=pivot, x='Сар', y='Дүн', hue='Данс код')

plt.title('Сар бүрийн дүн (данс тус бүрээр)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(data=monthly, x='Сар', y='Дүн')

plt.title('Сар бүрийн нийт дүн')
plt.xticks(rotation=45)
plt.ylabel('Дүн (₮)')
plt.xlabel('Сар')
plt.tight_layout()
plt.show()

In [12]:
import pandas as pd
df = pd.read_excel("Авлага_2023.03.17.xlsx", skiprows=5)
df=df.iloc[:197]
print(df.shape)
print(df.columns)
print(df.head(10))
print(df.info())

(197, 14)
Index(['120101 - Дансны авлага', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'MNT', 'Unnamed: 13'],
      dtype='object')
  120101 - Дансны авлага Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4  \
0                  10003        NaN     Соёлзул         NaN         NaN   
1                  10004        NaN     Билгүүн         NaN         NaN   
2                  10006        NaN   Цэнд-Аюуш         NaN         NaN   
3                  10007        NaN    Энхцэцэг         NaN         NaN   
4                  10009        NaN  Үүрийнтуяа         NaN         NaN   
5                   1001        NaN   Билгүүн.Г         NaN         NaN   
6                  10011        NaN   Бямбацогт         NaN         NaN   
7                  10020        NaN       Номин         NaN         NaN   
8                   1004        NaN   Цогжаргал         NaN        

In [13]:
print(df.isna().sum())

120101 - Дансны авлага      4
Unnamed: 1                194
Unnamed: 2                 22
Unnamed: 3                197
Unnamed: 4                197
Unnamed: 5                197
Unnamed: 6                  8
Unnamed: 7                194
Unnamed: 8                  8
Unnamed: 9                197
Unnamed: 10                 8
Unnamed: 11               197
MNT                         4
Unnamed: 13               197
dtype: int64


In [19]:
print(df["120101 - Дансны авлага"])




0      10003
1      10004
2      10006
3      10007
4      10009
       ...  
192     Нийт
193      NaN
194      NaN
195      NaN
196      NaN
Name: 120101 - Дансны авлага, Length: 197, dtype: object
